# Entrainement et fine-tuning du modèle
Annotations et prise en mains du fichier et des modules utilisés.
Modèle pré-enregistré utilisé: OpenAI Whisper
(plus récent et plus efficient que wav2vec2 de méta, voir https://huggingface.co/openai/whisper-small pour plus d'info)
Rq: l'abréviation RN sera utilisée pour Réseau Neuronal
https://huggingface.co/blog/fine-tune-whisper#prepare-feature-extractor-tokenizer-and-data
Whisper is pre-trained and fine-tuned using the cross-entropy objective function
Le projet ici est aussi basé sur le cours: https://huggingface.co/learn/audio-course/chapter5/fine-tuning

## 0/ Remarques quelconques
Seront rassemblées ici les remarques faites au cours de la prise en main du projet. Il faudrait y répondre ou en discuter avant la prise officielle du projet.
- limiter les audios à durer moins de 5sec, est ce que ce n'est pas trop court? en plus: The Whisper feature extractor performs two operations. It first pads/truncates a batch of audio samples such that all samples have an input length of 30s. Samples shorter than 30s are padded to 30s by appending zeros to the end of the sequence (zeros in an audio signal corresponding to no signal or silence). Samples longer than 30s are truncated to 30s.
- regarder si notebook_login() serait utile (https://huggingface.co/blog/fine-tune-whisper#prepare-feature-extractor-tokenizer-and-data)
- dans compute_metrics, pourquoi avoir fait le wer normalisé? cela apporte qch de vrmt intéressant ou pas?


In [2]:
# importation de TOUS les modules utilisés

# datasets est un module de hugging face et facilite le téléchargement et le pré-traitement des données issues de banques de données reconnues (dont MCVD)
from datasets import load_dataset, DatasetDict  # pour télécharger et mettre en forme le dataset  
from datasets import Audio                      # pour effectuer des opérations sur les fichiers audio (comme une modif de la fréquence d'enregistrement, sampling)

# transformers est un module de hugging face et fournit des centaines de modèles pré-entrainé. Celui qui nous interesse est le whisperprocessor (issu du papier: https://cdn.openai.com/papers/whisper.pdf)
# de ce que j'ai compris, le whisperProcessor n'est pas le modèle en soit, il s'agit plutot du module qui met en forme les données pour etre admissible par le modèle (mais où il est téléchargé ????)
# This processor pre-processes the audio to input features and tokenises the target text to labels.
# gère le padding/troncature, le log-mel, ainsi que la tokenization
from transformers import WhisperProcessor
# là, on importe vraiment le modèle whisper-small déjà pré-entrainé (on importe plutot la fonction qui importera le modèle pré-entrainé)
from transformers import WhisperForConditionalGeneration
# le module ci-dessous va permettre  de "normaliser" le txt, cad d'enlever les ponctuations et les accents
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import Seq2SeqTrainingArguments       # nrmlt, ca facilite l'entrainement du modèle (on l'utilise pr  faire passer les arg de l'entrainement)
from transformers import Seq2SeqTrainer                 # "fusionne"  les arg d'entrainement avec le modèle

# importation de pytorch, le module d'IA
import torch


# le module utilisé pour avoir la métrique d'évaluation (l'erreur)
import evaluate




# autres modules de python "annexes"
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from functools import partial


In [5]:
from huggingface_hub import notebook_login

notebook_login()

## I/ Création du dataset (issu de Mozilla Common Voice Dataset)
On cherche à fine-tuner le modèle déjà pré-entrainé. Pour cela, on utilise les données disponibles SLR.

In [6]:
common_voice = DatasetDict()

# téléchargement des données d'entrainement
# le fr spécifie que l'on prend QUE des données en francais
common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_6_0", "fr", split="train+validation", trust_remote_code=True
)   # on va chercher les données sur hugging_face (plateforme de mise en commun de données/modèles pour l'IA)
common_voice["test"] = load_dataset(
    "mozilla-foundation/common_voice_6_0", "fr", split="test", trust_remote_code=True
)

print(common_voice)

KeyboardInterrupt: 

In [7]:
common_voice = common_voice.select_columns(["audio", "sentence"])   # la BDD rassemble d'autres caractéristiques comme l'age, le genre, l'accent etc. cela ne nous interesse pas.

In [8]:
# on décide d'utiliser le modèle préentrainé whisper-small 3e sur les 6 proposés (en terme de taille). Le modèle contient 244M de paramètres
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="french", task="transcribe"
)
# le transcribe indique explicitement que l'on veut en fr-oral to fr-texte
# le modèle a probablement été entrainé de facon multi-linguistique => grande polyvalence (aussi disponible en english uniquement)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

c:\Users\maxen\anaconda3\envs\jcs_warlock\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maxen\.cache\huggingface\hub\models--openai--whisper-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [10]:
common_voice["train"].features

AttributeError: 'DatasetDict' object has no attribute 'features'

In [11]:
# The load_dataset function prepares audio samples with the sampling rate that they were published with. 
# This is not always the sampling rate expected by our model. In this case, we need to resample the audio to the correct sampling rate.
sampling_rate = processor.feature_extractor.sampling_rate           # le RN est créé pour une fréq d'enregistrement bien défini
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=sampling_rate))    # indique qu'il faut resampler les audios (mais ne les modifient pas, c'est "on the fly")

In [6]:
def prepare_dataset(batch):
    """
    Now we can write a function that takes a single training sample and passes it through the processor to prepare it for our model.
    source: https://huggingface.co/blog/audio-datasets
    """
    audio = batch["audio"]

    batch = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=batch["sentence"],
    )

    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return batch

In [7]:
common_voice.cleanup_cache_files()      # a quoi ca sert? jsp, pr clean des potentiels fichiers en cache???? ca n'a pas l'air très important en tous cas

{'train': 5, 'test': 0}

In [8]:
common_voice = common_voice.map(
    prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2
)
# https://huggingface.co/docs/datasets/process
# l'arg num_proc permet d'appliquer la fonction prepare_dataset en multiprocessing (ici en utilisant 2 coeur de proco/gpu)
# ATTENTION, visiblement ca peut merder entre linux et windows, cf https://discuss.huggingface.co/t/map-multiprocessing-issue/4085/12?page=2

Map (num_proc=2):   0%|          | 0/312847 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/15758 [00:00<?, ? examples/s]

In [9]:
# on veut enlever les ex audios qui font plus de 5 secondes! (pour eviter des pb de mémoire, ou des soucis de troncature)
max_input_length = 5.0


def is_audio_in_length_range(length):
    """
    retourne un booléen correspondant à "l'audio dure moins de 5 secondes"
    """
    return length < max_input_length

In [10]:
# on applique la fonction précédente avec un filtre (méthode filter). on garde que les audios qui font moins de 5 sec
# les 2 print permettent de comparer ce qui a été enlevé
print(common_voice["train"])
common_voice["train"] = common_voice["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)
print(common_voice["train"])

Filter:   0%|          | 0/312847 [00:00<?, ? examples/s]

In [12]:

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    """
    the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model
    les input_features (aka les audios en entrée sont déjà paddé/tronqué à 30s et sont déjà sous forme de log-mel) => il faut les transformer en pytorch tenseur
    par contre, les labels ne sont pas paddé => il faut les padder (on prend la longueur max et on pad avec des -100 pour que les 'ajouts' ne soient pas pris en compte 
    dans le calcul de l'erreur)
    """

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")   # ici ce n'est pas le pad qui ns interesse, c'est le return_tensor => on transforme en tensor

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)   # et on initialise le data collator que l'on vient de définir

In [14]:

# la métrique Word Error Rate WER est utilisée (c'est celle qui est utilisée communément pour les problèmes de reconnaissance vocale)
metric = evaluate.load("wer")

In [15]:

normalizer = BasicTextNormalizer()      # la fonction qui va enlever les accents et la ponctuation


def compute_metrics(pred):
    """
    calcule l'erreur entre ce qui a été prédit et ce qu'il fallait trouver
    se base sur l'erreur WER qui est communément utilisé dans les pb de RN audio
    renvoie 2 mesure, un WER "normal" et un WER du texte normalisé
    """
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    # se ref à https://huggingface.co/learn/audio-course/chapter5/evaluation
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [12]:
# et là, on télécharge le modèle RN pré-entrainé
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")     # on dl la version small (à changer potentiellement, dans tous les cas, c'est ce qui est communément utilisé)

model.safetensors:  53%|#####3    | 514M/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [13]:
# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="french", task="transcribe", use_cache=True
)   # ici, on indique au modèle qu'il parle francais, et qu'il doit faire du speech-to-text + qu'il a le droit d'utiliser la mémoire cache
# est equivalent à model.generation_config.task = "transcribe" (pr la partie transcribe)

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-fr",  # name on the HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-6,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    use_cpu=False
)

/home/paul-emile/anaconda3/envs/jcs-speech-to-text/lib/python3.12/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# on fusionne les arg d'entrainement avec le modèle (comme ca, on pourra faire un model.train après)
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

max_steps is given, it will override any value given in num_train_epochs


In [20]:
trainer.train()     # et on lance l'entrainement!!!

  0%|          | 0/500 [00:00<?, ?it/s]

/home/paul-emile/anaconda3/envs/jcs-speech-to-text/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 3.5579, 'grad_norm': 22.478580474853516, 'learning_rate': 2.5e-06, 'epoch': 0.01}
{'loss': 1.5979, 'grad_norm': 12.185086250305176, 'learning_rate': 5e-06, 'epoch': 0.02}
{'loss': 0.8312, 'grad_norm': 11.846023559570312, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.6308, 'grad_norm': 25.119054794311523, 'learning_rate': 5e-06, 'epoch': 0.04}
{'loss': 0.4488, 'grad_norm': 4.397257328033447, 'learning_rate': 5e-06, 'epoch': 0.05}
{'loss': 0.3396, 'grad_norm': 3.7539918422698975, 'learning_rate': 5e-06, 'epoch': 0.06}
{'loss': 0.3404, 'grad_norm': 4.50374174118042, 'learning_rate': 5e-06, 'epoch': 0.07}
{'loss': 0.328, 'grad_norm': 3.311716079711914, 'learning_rate': 5e-06, 'epoch': 0.08}
{'loss': 0.332, 'grad_norm': 3.4091596603393555, 'learning_rate': 5e-06, 'epoch': 0.1}
{'loss': 0.316, 'grad_norm': 3.599788188934326, 'learning_rate': 5e-06, 'epoch': 0.11}
{'loss': 0.3322, 'grad_norm': 3.4275434017181396, 'learning_rate': 5e-06, 'epoch': 0.12}
{'loss': 0.3178, 'grad_norm'

  0%|          | 0/493 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.3680204153060913, 'eval_wer_ortho': 27.943070455867836, 'eval_wer': 16.49266890799373, 'eval_runtime': 35081.463, 'eval_samples_per_second': 0.449, 'eval_steps_per_second': 0.014, 'epoch': 0.21}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


{'train_runtime': 160747.3349, 'train_samples_per_second': 0.199, 'train_steps_per_second': 0.003, 'train_loss': 0.5905919923782349, 'epoch': 0.21}


TrainOutput(global_step=500, training_loss=0.5905919923782349, metrics={'train_runtime': 160747.3349, 'train_samples_per_second': 0.199, 'train_steps_per_second': 0.003, 'total_flos': 9.23473281024e+18, 'train_loss': 0.5905919923782349, 'epoch': 0.21226915729144555})